# Analyzing a single speed skydive

In [11]:
from ssscoring import convertFlySight2SSScoring
from ssscoring import dropNonSkydiveDataFrom
from ssscoring import getSpeedSkydiveFrom
from ssscoring import isValidJump
from ssscoring import jumpAnalysisTable

import io

import bokeh.io as bi
import bokeh.plotting as bp
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd

# Initializations:
bp.output_notebook(hide_banner = True)
bi.curdoc().theme = 'dark_minimal'

## Get the data

In [12]:
uploader = widgets.FileUpload(description = 'Speed CSV', tooltip = 'FlySight speed file')
display(uploader)

FileUpload(value=(), description='Speed CSV', tooltip='FlySight speed file')

## Results

In [13]:
def processJump(data):
    data = data.copy()
    data = dropNonSkydiveDataFrom(data)
    window, data = getSpeedSkydiveFrom(data)
    validJump = isValidJump(data, window)
    score = 0.0
    
    if validJump:
        maxSpeed, table = jumpAnalysisTable(data)
        display(table)
        color = '#0f0'
        result = '🟢 valid'
        baseTime = data.iloc[0].timeUnix
        data['plotTime'] = data.timeUnix-baseTime
    
        scores = dict()  
        for spot in data.plotTime:
            r0 = data[data.plotTime == spot]
            r1 = data[data.plotTime == spot+3.0]
        
            if not r1.empty:
                scores[0.5*(float(r0.vKMh.iloc[0])+float(r1.vKMh.iloc[0]))] = spot
        score = max(scores)

    else:
        color = '#f00'
        result = '🔴 invalid'

    return (score, maxSpeed, scores, data, window, table, color, result)

In [14]:
data = convertFlySight2SSScoring(pd.read_csv(io.BytesIO(uploader.value[0].content), skiprows= (1,1)))
score, maxSpeed, scores, data, window, table, color, result = processJump(data)
validJumpStatus = '<hr><h1><span style="color: %s">%s jump; score = %.02f km/h</span></h1>' % (color, result, score)    

IndexError: tuple index out of range

In [ ]:
p = bp.figure(title = 'speed skydive',
              height = 500,
              width = 900,
              x_axis_label = 'seconds from exit',
              y_axis_label = 'km/h',
              x_range = (0,40),
              y_range = (0, 550))

In [ ]:
display(widgets.HTML(validJumpStatus))
if 'valid' in result:
    table.vKMh = table.vKMh.apply(round)
    table.hKMh = table.hKMh.apply(round)
    table['altitude (ft)'] = table['altitude (ft)'].apply(round)
    # table.drop(['altitude (ft)'], axis = 1, inplace = True)
    display(widgets.HTML('<h3>Max speed = {0:,.0f}; '.format(maxSpeed)+('exit at %d m (%d ft), breakoff at %d m (%d ft)</h3>'%(window.start, 3.2808*window.start, window.end, 3.2808*window.end))))
    table.index = ['']*len(table)
    display(table)
    p.line(data.plotTime, data.vKMh, legend_label = 'speed', line_width = 2, line_color = 'green')
    p.line(data.plotTime, data.hKMh, legend_label = 'H-speed', line_width = 2, line_color = 'red') # !!!
    p.segment(x0 = [ scores[score], scores[score]+3.0, ], y0 = [ 0.0, 0.0, ], x1 = [ scores[score], scores[score]+3.0, ], y1 = [ maxSpeed, maxSpeed, ])
    bp.show(p)